In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [2]:
# Load the dataset
file_path = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
data = pd.read_csv(file_path)


In [7]:
# Ensure all columns are numeric
data = data.apply(pd.to_numeric, errors='coerce')
data = data.dropna()  # Remove rows with NaN values

In [8]:
# Split into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [4]:
# Define the dataset class
class ForexDataset(Dataset):
    def __init__(self, data):
        self.features = torch.tensor(data.iloc[:, :-1].values, dtype=torch.float32)
        self.labels = torch.tensor(data.iloc[:, -1].values, dtype=torch.float32)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


In [6]:
# Create datasets and dataloaders
train_dataset = ForexDataset(train_data)
val_dataset = ForexDataset(val_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [ ]:
# Define the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, d_model, n_heads, n_layers, fc_dim, output_dim):
        super(TransformerModel, self).__init__()
        self.input_fc = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.fc = nn.Linear(d_model, fc_dim)
        self.output_fc = nn.Linear(fc_dim, output_dim)
    
    def forward(self, x):
        x = self.input_fc(x)
        x = self.transformer(x.unsqueeze(1)).squeeze(1)
        x = self.fc(x)
        return self.output_fc(x)


In [ ]:
# Hyperparameters
input_dim = train_dataset.features.shape[1]
d_model = 64
n_heads = 4
n_layers = 2
fc_dim = 32
output_dim = 1
lr = 0.001
epochs = 20

In [5]:
# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TransformerModel(input_dim, d_model, n_heads, n_layers, fc_dim, output_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


NameError: name 'TransformerModel' is not defined

In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for features, labels in val_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            loss = criterion(outputs.squeeze(), labels)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
